## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iqaluit,CA,63.7506,-68.5145,-27.87,97,86,4.47,overcast clouds
1,1,Busselton,AU,-33.6500,115.3333,64.09,61,92,17.98,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,72.48,58,46,1.70,scattered clouds
3,3,Kitimat,CA,54.0524,-128.6534,38.91,70,100,1.01,overcast clouds
4,4,Hilo,US,19.7297,-155.0900,77.79,64,69,4.00,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Busselton,AU,-33.6500,115.3333,64.09,61,92,17.98,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,72.48,58,46,1.70,scattered clouds
4,4,Hilo,US,19.7297,-155.0900,77.79,64,69,4.00,broken clouds
6,6,Geraldton,AU,-28.7667,114.6000,70.21,88,100,6.91,overcast clouds
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,74.59,92,92,7.83,overcast clouds
11,11,Waipawa,NZ,-41.4122,175.5153,71.92,64,69,8.59,broken clouds
14,14,Ponta Do Sol,PT,32.6667,-17.1000,65.12,39,1,9.64,clear sky
17,17,Atuona,PF,-9.8000,-139.0333,78.19,77,12,14.38,light rain
18,18,Bredasdorp,ZA,-34.5322,20.0403,69.12,88,87,9.22,overcast clouds
19,19,Rikitea,PF,-23.1203,-134.9692,76.86,70,100,5.06,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                797
City                   797
Country                785
Lat                    797
Lng                    797
Max Temp               797
Humidity               797
Cloudiness             797
Wind Speed             797
Current Description    797
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")
clean_df.count()

City_ID                785
City                   785
Country                785
Lat                    785
Lng                    785
Max Temp               785
Humidity               785
Cloudiness             785
Wind Speed             785
Current Description    785
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Busselton,AU,64.09,overcast clouds,-33.6500,115.3333,
2,Mataura,NZ,72.48,scattered clouds,-46.1927,168.8643,
4,Hilo,US,77.79,broken clouds,19.7297,-155.0900,
6,Geraldton,AU,70.21,overcast clouds,-28.7667,114.6000,
9,Arraial Do Cabo,BR,74.59,overcast clouds,-22.9661,-42.0278,
11,Waipawa,NZ,71.92,broken clouds,-41.4122,175.5153,
14,Ponta Do Sol,PT,65.12,clear sky,32.6667,-17.1000,
17,Atuona,PF,78.19,light rain,-9.8000,-139.0333,
18,Bredasdorp,ZA,69.12,overcast clouds,-34.5322,20.0403,
19,Rikitea,PF,76.86,overcast clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))